## Rental Prices in Brussels (Belgium)
### Data Cleaning

In this project, we will be working with a dataset on rental prices in Brussels, Belgium. The dataset contains information on rental prices for apartments in Brussels, Belgium. The dataset was originally published on Kaggle by the user "Rafael" and can be found [here](https://www.kaggle.com/rafjaa/residential-real-estate-prices-in-brussels).


In [1]:
import pandas as pd
import re

In [2]:
# Read the CSV file into a DataFrame
rental_df_raw = pd.read_csv('rental_df_dirty.csv')
rental_df = rental_df_raw.copy()

In [3]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Print the DataFrame
rental_df.head()

,Available date,Floor,Kitchen type,Bedrooms,Dressing room,Bathrooms,Toilets,Office,Professional space,Small pet-friendly,Armored door,Primary energy consumption,Energy class,Reference number of the EPC report,CO₂ emission,Yearly theoretical total energy consumption,Heating type,Heat pump,Photovoltaic solar panels,Thermic solar panels,Common water heater,Double glazing,Type of building,Monthly rental price,Monthly costs,Tenement building,price,space,address,listing_type,Number of floors,Number of frontages,Outdoor parking spaces,Living area,Living room surface,Bedroom 1 surface,Bedroom 2 surface,Furnished,Terrace surface,Intercom,Visio phone,As built plan,Address,Website,External reference,agency,Construction year,Building condition,Living room,Dining room,Laundry room,Basement,Terrace orientation,Caretaker,Elevator,Accessible for disabled people,Secure access / alarm,Air conditioning,TV cable,Jacuzzi,Sauna,Swimming pool,Internet,Conformity certification for fuel tanks,Possible priority purchase right,Available as of,Street frontage width,description,Kitchen surface,Basement surface,Neighbourhood or locality,How many fireplaces?,Terrace,Garden surface,Garden orientation,Planning permission obtained,Subdivision permit,Shower rooms,Flood zone type,Surroundings type,Virtual visit,Property name,Covered parking spaces,Proceedings for breach of planning regulations,Professional space surface,Obligation to build,Attic surface,Bedroom 3 surface,Office surface,Attic,Surface of the plot,Connection to sewer network,"Gas, water & electricity",Total ground floor buildable,Latest land use designation,Garden,E-level (overall energy performance),Agent's name,Land is facing street,Wooded land,Plot at rear,Flat land,Bedroom 4 surface,EPC description,Bedroom 5 surface,Width of the lot on the street,Isolated,Extra information,Number of annexes
0,January 31 2023 - 12:00 AM,2.0,USA installed,1.0,No,1.0,1.0,No,No,No,No,Not specified,Not specified,Not specified,Not specified,Not specified,Gas,No,No,No,No,Yes,All kind,€ 750\n750 €,€ 30\n30€,No,€750 (+ €30)\n750€ + 30€ per month,1 bedroom,1190 — Forest,Apartment for rent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,March 1 2023 - 12:00 AM,3.0,Installed,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,Not specified,Not specified,Not specified,Not specified,Not specified,Gas,No,No,No,NaN,No,NaN,"€ 1,150\n1150 €",€ 200\n200€,No,"€1,150 (+ €200)\n1150€ + 200€ per month",2 bedrooms | 90 m²\nsquare meters,Ask for the exact address,Apartment for rent,3.0,2.0,1.0,90 m²\nsquare meters,30 m²\nsquare meters,16 m²\nsquare meters,12 m²\nsquare meters,No,10 m²\nsquare meters,Yes,Yes,No,Rue Des Carmélites 88\n1180 - Uccle,http://www.immofadan.be,15274JADS,IMMO FADAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,April 1 2023 - 12:00 AM,3.0,USA hyper equipped,2.0,No,1.0,1.0,No,No,No,No,242 kWh/m²\nkilowatt hour per square meters,E,20130605-0000169186-01-4,52 kg CO₂/m²,Not specified,Gas,No,No,No,No,Yes,All kind,"€ 1,200\n1200 €",€ 80\n80€,No,"€1,200 (+ €80)\n1200€ + 80€ per month",2 bedrooms | 90 m²\nsquare meters,1040 — Etterbeek,Apartment for rent,3.0,2.0,NaN,90 m²\nsquare meters,NaN,16 m²\nsquare meters,10 m²\nsquare meters,NaN,3 m²\nsquare meters,Yes,No,NaN,NaN,NaN,NaN,NaN,1970.0,Good,Yes,Yes,Yes,Yes,South East,No,No,No,No,No,Yes,No,No,No,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.0,Installed,2.0,NaN,1.0,1.0,NaN,N

In [4]:
rental_df.shape

(3477, 109)

In [5]:
null_proportions = rental_df.isnull().mean()
null_proportions

Available date                    0.438596
Floor                             0.241875
Kitchen type                      0.182053
Bedrooms                          0.047167
Dressing room                     0.728502
                                    ...   
Bedroom 5 surface                 0.968364
Width of the lot on the street    0.997412
Isolated                          0.997699
Extra information                 0.997412
Number of annexes                 0.998274
Length: 109, dtype: float64

In [6]:
columns_to_drop = null_proportions[null_proportions > 0.70].index
columns_to_drop

Index(['Dressing room', 'Professional space', 'Small pet-friendly',
       'Photovoltaic solar panels', 'Common water heater', 'Type of building',
       'Outdoor parking spaces', 'As built plan', 'Living room', 'Dining room',
       'Laundry room', 'Terrace orientation', 'Caretaker', 'Air conditioning',
       'Jacuzzi', 'Sauna', 'Internet',
       'Conformity certification for fuel tanks',
       'Possible priority purchase right', 'Street frontage width',
       'Basement surface', 'Neighbourhood or locality', 'How many fireplaces?',
       'Terrace', 'Garden surface', 'Garden orientation',
       'Planning permission obtained', 'Subdivision permit', 'Flood zone type',
       'Virtual visit', 'Property name', 'Covered parking spaces',
       'Proceedings for breach of planning regulations',
       'Professional space surface', 'Obligation to build', 'Attic surface',
       'Bedroom 3 surface', 'Office surface', 'Attic', 'Surface of the plot',
       'Connection to sewer network', 'G

In [7]:
# Drop the columns with a proportion of null values greater than 75%
rental_df.drop(columns_to_drop, axis=1, inplace=True)
rental_df.shape

(3477, 51)

In [8]:
# Drop the columns that we found irrelevant
rental_df.drop(["Available date", "Reference number of the EPC report", "space", 
                "Yearly theoretical total energy consumption", "Website",
                "External reference", "description"], axis=1, inplace=True)

In [9]:
# Create a copy of the DataFrame
df = rental_df.copy()

In [10]:
# Create functions to split and clean the data
extract_numbers = lambda x: int(re.findall('\d+', str(x))[0]) if re.findall('\d+', str(x)) else None
extract_price= lambda x: int(str(x).split('\n')[0].replace('€', '').replace(' ', '').replace(',', '')) if isinstance(x, str) else None

In [11]:
# Get the prices in cleaned format
df['Monthly rental price'] = df['Monthly rental price'].apply(extract_price)
df['Monthly costs'] = df['Monthly costs'].apply(extract_price)

# Calculate the price
df['price'] = df['Monthly rental price']+df['Monthly costs']

In [12]:
# Get the surfaces in cleaned format
surfaces = ["Primary energy consumption", "CO₂ emission", "Living area", "Living room surface", 
            "Bedroom 1 surface", "Bedroom 2 surface", "Terrace surface", "Kitchen surface"]

df[surfaces]=df[surfaces].applymap(extract_numbers)

In [13]:
# Get the addresses in cleaned format
df["address"] = df['address'].apply(extract_numbers)

# Define a list of valid postal codes for Brussels
valid_postal_codes = [1000, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1081,
                      1082, 1083, 1090, 1120, 1130, 1140, 1150, 1160, 1170,
                      1180, 1190, 1200, 1210]

# Create a boolean mask for rows with invalid postal codes
invalid_mask = ~(df['address'].isin(valid_postal_codes) | df['address'].isna())

# Drop the rows with invalid postal codes
df = df.drop(df[invalid_mask].index)

In [14]:
# Define the regular expression pattern to match postal codes
postal_code_pattern = r'\b\d{4}\b'

# Define a function to extract postal codes from addresses
def extract_postal_code(address):
    match = re.search(postal_code_pattern, str(address))
    if match:
        return int(match.group(0))
    else:
        return None

# Apply the extract_postal_code function to the "Address" column and create a new column "Zip_Code"
df['Zip_Code'] = df['Address'].apply(extract_postal_code)
df.drop('Address', axis=1, inplace=True)

In [15]:
# Define a list of valid postal codes for Brussels
valid_postal_codes = [1000, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1081, 
                      1082, 1083, 1090, 1120, 1130, 1140, 1150, 1160, 1170, 
                      1180, 1190, 1200, 1210]

# Create a boolean mask for rows with invalid postal codes or missing values
invalid_mask = ~(df['Zip_Code'].isin(valid_postal_codes) | df['Zip_Code'].isna())

# Drop the rows with invalid postal codes
df = df.drop(df[invalid_mask].index)

In [16]:
# Get the final 'Zip_Code' column cleaned and put in the right format
df['Zip_Code'].fillna(df['address'], inplace=True)
df.drop('address', axis=1, inplace=True)
df.dropna(subset=['Zip_Code'], inplace=True)
df['Zip_Code'] = df['Zip_Code'].astype(int)
df['Zip_Code'] = df['Zip_Code'].astype(str)

In [17]:
# Adding "State_Name" column to the DataFrame
# Create and convert a dictionary to a dataframe to merge the Zip_Code code and the states name
brussels_postcodes = {
  "1000": "Bruxelles-Ville", "1020": "Laeken", "1030": "Schaerbeek", "1040": "Etterbeek",
  "1050": "Ixelles", "1060": "Saint-Gilles", "1070": "Anderlecht", "1080": "Molenbeek-Saint-Jean",
  "1081": "Koekelberg", "1082": "Berchem-Sainte-Agathe", "1083": "Ganshoren", "1090": "Jette",
  "1120": "Neder-Over-Heembeek", "1130": "Haren", "1140": "Evere", "1150": "Woluwe-Saint-Pierre",
  "1160": "Auderghem", "1170": "Watermael-Boitsfort", "1180": "Uccle", "1190": "Forest", "1200": "Woluwe-Saint-Lambert"
  }
zip_code = pd.DataFrame(list(brussels_postcodes.items()), columns=['Zip_Code', 'State_Name'])

# Merge the DataFrames on the "Zip_Code" column
df = pd.merge(df, zip_code, on='Zip_Code')

In [18]:
df

,Floor,Kitchen type,Bedrooms,Bathrooms,Toilets,Office,Armored door,Primary energy consumption,Energy class,CO₂ emission,Heating type,Heat pump,Thermic solar panels,Double glazing,Monthly rental price,Monthly costs,Tenement building,price,listing_type,Number of floors,Number of frontages,Living area,Living room surface,Bedroom 1 surface,Bedroom 2 surface,Furnished,Terrace surface,Intercom,Visio phone,agency,Construction year,Building condition,Basement,Elevator,Accessible for disabled people,Secure access / alarm,TV cable,Swimming pool,Available as of,Kitchen surface,Shower rooms,Surroundings type,Zip_Code,State_Name
0,2.0,USA installed,1.0,1.0,1.0,No,No,NaN,Not specified,NaN,Gas,No,No,Yes,750.0,30.0,No,780.0,Apartment for rent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1190,Forest
1,3.0,Installed,1.0,NaN,1.0,No,No,NaN,Not specified,NaN,Gas,No,No,Yes,950.0,NaN,No,NaN,Studio for rent,3.0,2.0,45.0,NaN,20.0,NaN,Yes,8.0,Yes,Yes,NaN,NaN,Just renovated,NaN,No,No,No,Yes,No,Immediately,15.0,1.0,Landscape,1190,Forest
2,3.0,Hyper equipped,2.0,1.0,1.0,No,No,NaN,Not specified,NaN,Gas,No,No,Yes,1950.0,NaN,No,NaN,Penthouse for rent,3.0,2.0,110.0,15.0,16.0,10.0,Yes,40.0,Yes,Yes,NaN,2015.0,Just renovated,NaN,No,No,No,Yes,No,Immediately,20.0,NaN,Landscape,1190,Forest
3,2.0,Installed,1.0,NaN,1.0,No,No,NaN,Not specified,NaN,Gas,No,No,Yes,750.0,NaN,No,NaN,Apartment for rent,3.0,2.0,54.0,15.0,14.0,NaN,NaN,NaN,Yes,No,NaN,2009.0,As new,NaN,No,No,No,No,No,Immediately,6.0,1.0,NaN,1190,Forest
4,3.0,Hyper equipped,2.0,1.0,1.0,NaN,NaN,240.0,E,NaN,Gas,NaN,NaN,Yes,835.0,NaN,No,NaN,Apartment for rent,4.0,NaN,80.0,35.0,NaN,NaN,Yes,12.0,NaN,NaN,Century 21 A à Z Immobilier,NaN,As new,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1190,Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,NaN,USA semi equipped,2.0,1.0,3.0,NaN,Yes,32.0,A,4.0,Gas,NaN,Yes,Yes,2000.0,150.0,No,2150.0,Apartment for rent,3.0,4.0,124.0,35.0,12.0,16.0,No,73.0,NaN,Yes,Sylver Properties,2022.0,As new,Yes,Yes,NaN,NaN,Yes,NaN,Immediately,9.0,1.0,Urban,1082,Berchem-Sainte-Agathe
2759,1.0,USA installed,2.0,1.0,1.0,NaN,Yes,NaN,Not specified,NaN,Gas,NaN,NaN,Yes,975.0,75.0,No,1050.0,Apartment for rent,NaN,2.0,80.0,25.0,14.0,9.0,NaN,11.0,NaN,Yes,Sylver Properties,2022.0,As new,Yes,Yes,Yes,NaN,Yes,NaN,Immediately,5.0,NaN,NaN,1082,Berchem-Sainte-Agathe
2760,1.0,NaN,2.0,1.0,1.0,No,No,133.0,C,28.0,Gas,No,No,Yes,1295.0,90.0,No,1385.0,Apartment for rent,4.0,3.0,121.0,44.0,24.0,18.0,NaN,14.0,No,Yes,NaN,2004.0,As new,NaN,Yes,No,Yes,Yes,No,NaN,11.0,NaN,Landscape,1082,Berchem-Sainte-Agathe
2761,4.0,NaN,3.0,1.0,1.0,NaN,NaN,335.0,F,NaN,Gas,NaN,NaN,Yes,1200.0,NaN,No,NaN,Apartment for rent,4.0,NaN,130.0,25.0,11.0,9.0,No,NaN,NaN,NaN,CENTURY 21 Invest,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,Immediately,24.0,NaN,NaN,1082,Berchem-Sainte-Agathe


In [19]:
# Save the final dataset cleaned
df.to_csv('rental_data_clean.csv', index=False)